# SCGNN Test - Trafik - PCA

In [ ]:
import pandas as pd
import sys
from pathlib import Path
sys.path.append('../..')
from src import config
from src.utils.dataset import TrafficDataset
from src.utils.model import TrafficPredictor
from torch.utils.data import DataLoader

In [ ]:
df = pd.read_parquet(config.DATA_INTERIM / 'sample.parquet')
df = df.fillna(0)
X_STEP, Y_STEP = 2, 1
data_array = df.values

In [ ]:
PCA_COMPONENTS = 5
dataset = TrafficDataset(data_array, X_STEP, Y_STEP, pca_components=PCA_COMPONENTS)
sensors = dataset.sensors
edges = []
for i in range(sensors - 1):
    edges.append([i, i + 1])
    edges.append([i + 1, i])
edge_index = torch.tensor(edges, dtype=torch.long).t()
predictor = TrafficPredictor('scgnn', sensors, X_STEP, Y_STEP, edge_index=edge_index, pca_model=dataset.pca if True else None)
predictor.load_model(config.PROJECT_ROOT.joinpath('models/scgnn_trafik_pca.pt'))
loader = DataLoader(dataset, batch_size=10, num_workers=8)
preds = predictor.inference(loader)
preds[:5]